In [1]:
!pip install -q optuna

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, f1_score
import xgboost as xgb
import optuna
import joblib

In [ ]:
# Load the dataset  
df = pd.read_csv("/content/drive/MyDrive/Project_CV/Telco-Customer-Churn.csv")

In [ ]:
# Drop unnecessary columns  
df.drop("customerID", axis=1, inplace=True)

# Clean data in the TotalCharges column  
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df.dropna(inplace=True)

# Encode the target column  
df['Churn'] = df['Churn'].map({'No': 0, 'Yes': 1})

# Encode categorical features  
for col in df.select_dtypes(include='object').columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

In [ ]:
# Split the data  
X = df.drop('Churn', axis=1).reset_index(drop=True)
y = df['Churn'].reset_index(drop=True)

In [ ]:
# Perform Stratified K-Fold  
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# Optuna hyperparameter tuning function  
def objective(trial):
    params = {
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "n_estimators": trial.suggest_int("n_estimators", 100, 300),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0)
    }

    scores = []

    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # Skip if data does not contain both classes 
        if len(np.unique(y_train)) < 2 or len(np.unique(y_test)) < 2:
            print(f"Skipping fold {fold+1}: only one class in y_train or y_test")
            continue

        model = xgb.XGBClassifier(**params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_train, y_train), (X_test, y_test)],
            verbose=False
        )

        evals_result = model.evals_result()
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        scores.append(acc)

        print(f"Fold {fold+1} - Train logloss: {evals_result['validation_0']['logloss'][-1]:.4f}, "
              f"Train error: {evals_result['validation_0']['error'][-1]:.4f}, "
              f"Val logloss: {evals_result['validation_1']['logloss'][-1]:.4f}, "
              f"Val error: {evals_result['validation_1']['error'][-1]:.4f}")

    return np.mean(scores) if scores else 0.0

In [ ]:
# Perform tuning with Optuna  
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

[I 2025-07-07 16:26:48,971] A new study created in memory with name: no-name-395d1d7d-c902-4f15-a3f5-aaea6f45baf6


Fold 1 - Train logloss: 0.4109, Train error: 0.1952, Val logloss: 0.4193, Val error: 0.1912
Fold 2 - Train logloss: 0.4123, Train error: 0.1934, Val logloss: 0.4151, Val error: 0.1969
Fold 3 - Train logloss: 0.4104, Train error: 0.1927, Val logloss: 0.4259, Val error: 0.2148
Fold 4 - Train logloss: 0.4103, Train error: 0.1939, Val logloss: 0.4204, Val error: 0.1956


[I 2025-07-07 16:26:53,964] Trial 0 finished with value: 0.8011931806118767 and parameters: {'max_depth': 3, 'learning_rate': 0.021555907680362137, 'n_estimators': 131, 'subsample': 0.8456422217582689, 'colsample_bytree': 0.6300713599672738}. Best is trial 0 with value: 0.8011931806118767.


Fold 5 - Train logloss: 0.4099, Train error: 0.1973, Val logloss: 0.4252, Val error: 0.1956
Fold 1 - Train logloss: 0.2297, Train error: 0.0884, Val logloss: 0.4688, Val error: 0.2082
Fold 2 - Train logloss: 0.2258, Train error: 0.0875, Val logloss: 0.4813, Val error: 0.2175
Fold 3 - Train logloss: 0.2264, Train error: 0.0859, Val logloss: 0.4785, Val error: 0.2297
Fold 4 - Train logloss: 0.2263, Train error: 0.0864, Val logloss: 0.4801, Val error: 0.2176


[I 2025-07-07 16:26:56,733] Trial 1 finished with value: 0.7819951249644888 and parameters: {'max_depth': 4, 'learning_rate': 0.2634773093470411, 'n_estimators': 266, 'subsample': 0.7019316820890591, 'colsample_bytree': 0.9706429566008548}. Best is trial 0 with value: 0.8011931806118767.


...

Fold 5 - Train logloss: 0.1436, Train error: 0.0238, Val logloss: 0.4845, Val error: 0.2048
Fold 1 - Train logloss: 0.2921, Train error: 0.1285, Val logloss: 0.4372, Val error: 0.2061
Fold 2 - Train logloss: 0.2922, Train error: 0.1264, Val logloss: 0.4305, Val error: 0.2139
Fold 3 - Train logloss: 0.2929, Train error: 0.1276, Val logloss: 0.4415, Val error: 0.2191
Fold 4 - Train logloss: 0.2899, Train error: 0.1257, Val logloss: 0.4350, Val error: 0.2134


[I 2025-07-07 16:28:28,305] Trial 28 finished with value: 0.789248939209662 and parameters: {'max_depth': 5, 'learning_rate': 0.13408322332582892, 'n_estimators': 140, 'subsample': 0.5446880237721701, 'colsample_bytree': 0.7247243247912222}. Best is trial 22 with value: 0.80460782856698.


Fold 5 - Train logloss: 0.2926, Train error: 0.1290, Val logloss: 0.4458, Val error: 0.2013
Fold 1 - Train logloss: 0.3109, Train error: 0.1339, Val logloss: 0.4208, Val error: 0.1983
Fold 2 - Train logloss: 0.3147, Train error: 0.1388, Val logloss: 0.4102, Val error: 0.1947
Fold 3 - Train logloss: 0.3121, Train error: 0.1369, Val logloss: 0.4200, Val error: 0.2091
Fold 4 - Train logloss: 0.3105, Train error: 0.1392, Val logloss: 0.4219, Val error: 0.2020


[I 2025-07-07 16:28:30,229] Trial 29 finished with value: 0.8014783833322717 and parameters: {'max_depth': 7, 'learning_rate': 0.038674670924613186, 'n_estimators': 118, 'subsample': 0.6081933836761709, 'colsample_bytree': 0.778918941641634}. Best is trial 22 with value: 0.80460782856698.


Fold 5 - Train logloss: 0.3142, Train error: 0.1358, Val logloss: 0.4230, Val error: 0.1885


In [ ]:
# Save the final model with the best parameters 
best_params = study.best_params
best_params.update({
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"]
})

print("\nBest parameters found by Optuna:")
print(best_params)




Best parameters found by Optuna:
{'max_depth': 6, 'learning_rate': 0.03877299918289841, 'n_estimators': 127, 'subsample': 0.5527172291879321, 'colsample_bytree': 0.7226512598943868, 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error']}


In [ ]:
# Re-evaluate the model  
f1_scores, acc_scores, prec_scores = [], [], []
models = []

for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    if len(np.unique(y_train)) < 2 or len(np.unique(y_test)) < 2:
        print(f"Skipping final fold {fold+1}: only one class present.")
        continue

    model = xgb.XGBClassifier(**best_params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=False
    )

    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred, average='macro')
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='macro')

    f1_scores.append(f1)
    acc_scores.append(acc)
    prec_scores.append(prec)
    models.append(model)

    print(f"Fold {fold+1} - F1 Score: {f1:.4f}, Accuracy: {acc:.4f}, Precision: {prec:.4f}")

# Find the best model index based on F1-score  
best_index = np.argmax(f1_scores)
best_model = models[best_index]

# Save the best model to file  
joblib.dump(best_model, f"/content/drive/MyDrive/Project_CV/xgb_model_best_fold{fold+1}.pkl")
print(f"\nModel terbaik disimpan dari fold {best_index + 1} dengan F1 Score: {f1_scores[best_index]:.4f}")


Fold 1 - F1 Score: 0.7321, Accuracy: 0.8074, Precision: 0.7598
Fold 2 - F1 Score: 0.7226, Accuracy: 0.7996, Precision: 0.7472
Fold 3 - F1 Score: 0.7208, Accuracy: 0.7959, Precision: 0.7400
Fold 4 - F1 Score: 0.7268, Accuracy: 0.8037, Precision: 0.7544
Fold 5 - F1 Score: 0.7441, Accuracy: 0.8165, Precision: 0.7746

Model terbaik disimpan dari fold 5 dengan F1 Score: 0.7441


In [ ]:
# Display the results  
hasil = pd.DataFrame({
    'Fold': list(range(1, len(f1_scores)+1)),
    'F1 Score (macro)': f1_scores,
    'Accuracy (macro)': acc_scores,
    'Precision (macro)': prec_scores
})

print("\n=== Evaluasi Model per Fold ===")
print(hasil)


=== Evaluasi Model per Fold ===
   Fold  F1 Score (macro)  Accuracy (macro)  Precision (macro)
0     1          0.732072          0.807392           0.759811
1     2          0.722646          0.799574           0.747153
2     3          0.720822          0.795875           0.739961
3     4          0.726783          0.803698           0.754414
4     5          0.744063          0.816501           0.774573


In [ ]:
# Display the overall average metrics  
mean_metrics = hasil[['F1 Score (macro)', 'Accuracy (macro)', 'Precision (macro)']].mean()
print("\n=== Rata-rata Keseluruhan ===")
print(mean_metrics)


=== Rata-rata Keseluruhan ===
F1 Score (macro)     0.729277
Accuracy (macro)     0.804608
Precision (macro)    0.755182
dtype: float64
